Installing packages


In [36]:
!pip install statsmodels
#import needed packages
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from google.colab import drive
#mounting my google driving to access data
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Creatings Pandas data frames

In [37]:
#converting the csv synthetic_data to a pandas data frame
df = pd.read_csv('/content/drive/MyDrive/Second Year Project/Student Data /2021-2228_anon_cleaned.csv')

#converting the xlsx data to a pandas data frame
df2 = pd.read_excel("/content/drive/MyDrive/Second Year Project/Student Data /Master LA _ Mentor Database.xlsx")
df2 = df2.rename(columns={'term': 'Term'})

<ipython-input-37-07481f5bd3e4>:2: DtypeWarning: Columns (7,8,9,10,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/Second Year Project/Student Data /2021-2228_anon_cleaned.csv')


Below I am filtering the data to only look at coures starting in the Fall of 2013 and excluding Summer terms.

In [38]:
df = df[(df['Term'] >= 2131) & (df['Term'] != 2133)]
df = df[df['Term'] % 10 != 8]
#df = df[(2131 <= df["Cohort Term"]) & (df["Cohort Term"] <= 2221 - 60)]


Below I am creating a six year graduation rate SYG for all of the students where if a student graduated with in six years they get a 1 and if not they get a 0.

In [39]:
df["SYG"] = 0
df.loc[(df["Degree Term"] < df["Cohort Term"]+60) & ((df["Degree Term"] >= df["Cohort Term"])), "SYG"] = 1
print('Overall SYG rate:', len(df[df['SYG'] == 1]) / len(df['SYG']))



<ipython-input-39-813bc6847717>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["SYG"] = 0


Overall SYG rate: 0.5398157203891492


Below I am filtering out all data before 2020 to account for the change in grading and class structure during the COVID-19 pandemic

In [40]:
df = df[df['Term'] <= 2195]

Adding in Learning Assistant Column where 1 is a LA and 0 is no LA

In [41]:
#creating a function to find all of the
def LA_Finder(subject, catalog_nbr, class_section, Term, df):
    df.loc[(df['Term'] == Term) & (df['subject'] == subject) &
           (df['class_section'] == class_section) & (df['catalog_nbr'] == catalog_nbr), 'LA'] = 1
    return df

df['LA'] = 0

# looping over alll the rows in the LA dataframe
for index, row in df2.iterrows():
    subject = row['subject']
    catalog_nbr = row['catalog_nbr']
    class_section = row['class_section']
    Term = row['Term']
    df = LA_Finder(subject, catalog_nbr, class_section, Term, df)


<ipython-input-41-b014f0120561>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['LA'] = 0


Below I am adding an LA count feater to look at the number of LAs a student had before taking a course. This is so we can see if previous LA support for a student is helping them in the current class.

In [42]:
#Calculating the cumlative sum of 'LA' for each student (grouped by 'ID') over time (sorted by 'Term')
df['LA Count'] = df.sort_values(by='Term').groupby('ID')['LA'].cumsum()

#Subrtrqct 1 from each element in the 'LA Count' column, but keeing zeros unchaged
df['LA Count'] = df['LA Count'].apply(lambda x: x-1 if x > 0  else x)

I am narrowing down the data to only look at University Physics I and II and Multivariable Calculus

In [43]:

df = df[((df['subject'] == 'PHYS') & ((df['catalog_nbr'] == '211') | (df['catalog_nbr'] == '212'))) |
        ((df['subject'] == 'MATH') & (df['catalog_nbr'] == '211'))]
        # ((df['subject'] == 'MATH') & ((df['catalog_nbr'] == '171') | (df['catalog_nbr'] == '211')))]

Below I am calculating the SYG rate for students who had an LA at some points.

In [44]:
LA_df = df[df['LA'] == 1]
print('Overall SYG rate with an LA:',len(LA_df[LA_df['SYG'] == 1]) / len(LA_df['SYG']))

NO_LA_df = df[df['LA'] == 0]
print('Overall SYG rate without an LA:',len(NO_LA_df[NO_LA_df['SYG'] == 1]) / len(NO_LA_df['SYG']))


Overall SYG rate with an LA: 0.7243574628625324
Overall SYG rate without an LA: 0.7141181644920099


Adding in binary columns to capture various demographics

In [45]:
# Create a new column 'gender dummy' with 1 for for female/other or 0 for male
df['gender dummy'] = df['gender'].apply(lambda x: 1 if x in ['F', 'U'] else 0)

# Create a new column 'First Gen dummy' with 1 for First Generation students or 0 for else
df['First Gen dummy'] = df['First Generation'].apply(lambda x: 1 if x == 'First Gen' else 0)
# Fill NaN values with 0
df['First Generation'] = df['First Generation'].fillna(0)

# Create a new column 'AALANA dummy' with 1 for AALANA or 0 for else
df['AALANA dummy'] = df['AALANA'].apply(lambda x: 1 if x == 'AALANA' else 0)

# Create a new column 'DFW dummy' with 1 for 'D' or 'F' or 'W' grade or lower or 0 for else
df['DFW dummy'] = df['Grade'].apply(lambda x: 1 if x in ['D', 'F', 'W'] else 0)


Below I am calculating the DFW rate for students who took UP1, UP2, CP1, or, CP2 with and without an LA

In [46]:
print('Overall DFW rate:', len(df[df['DFW dummy'] == 1]) / len(df))



LA_df = df[df['LA'] == 1]
print('Overall DFW rate only classes with LA:',len(LA_df[LA_df['DFW dummy'] == 1]) / len(LA_df))

NO_LA_df = df[df['LA'] == 0]
print('Overall DFW rate only classes without LA:',len(NO_LA_df[NO_LA_df['DFW dummy'] == 1]) /  len(NO_LA_df))

Overall DFW rate: 0.2339540770204303
Overall DFW rate only classes with LA: 0.18698420183918887
Overall DFW rate only classes without LA: 0.2631578947368421


In [47]:
# Grouping by 'catalog_nbr', 'class_section', and 'Term' and counting occurrences
counts = df.groupby(['catalog_nbr', 'class_section', 'Term']).size().reset_index(name='count')

# Filtering out the rows with 5 or fewer occurrences
#filtered_counts = counts[counts['count'] > 5]

# Filtering the original DataFrame based on the filtered counts
#df = df.merge(filtered_counts, on=['catalog_nbr', 'class_section', 'Term'])

# # Filtering out rows where 'class_section' contains in the string
df = df[~(df['section Type'] == 'LAB')]
df = df[(df['class_section'] != '600') & (df['class_section'] != '601')]


In [48]:
counts = df.groupby(['catalog_nbr', 'class_section', 'Term']).size().reset_index(name='count')

# Sorting the DataFrame by 'count' column from smallest to largest
counts_sorted = counts.sort_values(by='count')


counts_sorted

,catalog_nbr,class_section,Term,count
122,211,11,2131.0,5
131,211,14,2161.0,7
128,211,13,2151.0,9
248,212,12,2195.0,9
259,212,14,2155.0,9
...,...,...,...,...
50,211,04,2185.0,81
40,211,04,2135.0,82
44,211,04,2155.0,84
48,211,04,2175.0,85


In [49]:
len(df)

10675

In [50]:
# Set the file name
file_name = "HLM_Data_For_DFW_With_LA_Count.csv"


# Set the path to save the CSV file in your Google Drive
file_path = f'/content/drive/MyDrive/{file_name}'

# Save DataFrame to CSV
df.to_csv(file_path, index=False)

# Display the path where the file is saved

In [51]:
# # Rename the 'gender' column to 'Gender'
# df = df.rename(columns={'gender': 'Gender'})
# # Map values in the 'Gender' column
# gender_mapping = {0: 'Male', 1: 'Non Male'}
# df['gender dummy'] = df['gender dummy'].map(gender_mapping)

# df = df.rename(columns={'AALANA': 'Race'})
# # Map values in the 'Race' column
# df['Race'] = df['Race'].apply(lambda x: 'AALANA' if x == 'AALANA' else 'Non AALANA')

# # Rename the 'First Generation' column to 'First Gen status'
# df = df.rename(columns={'First Generation': 'First Gen status'})
# # Map values in the 'First Gen status' column
# df['First Gen status'] = df['First Gen status'].apply(lambda x: 'FG' if x == 'First Gen' else 'CG')


# # Rename the 'LA' column to 'Instructional type'
# df = df.rename(columns={'LA': 'Instruction type'})
# # Map values in the 'Instructional type' column
# df['Instruction type'] = df['Instruction type'].apply(lambda x: 'LA' if x == 1 else 'Traditional')

In [52]:
# # Create a cross-tabulation table
# table = pd.crosstab(index=[df['gender dummy'], df['Race'], df['First Gen status'], df['Instruction type']],
#                     columns='N',
#                     margins=True, margins_name='Total', rownames=['gender dummy', 'Race', 'First Gen status', 'Instruction type'],
#                     colnames=['N'])

# # Convert the table to LaTeX format
# latex_table = table.to_latex()

# # Print the LaTeX table
# print(latex_table)

# #print(table)


In [53]:
df['subject'].value_counts()

len(df)

10675